## Lecture 9: Image Recognition 
In this project a combination of different techniques such as SVD, wavelet transform and 
statistical techniques such as LDA (linear discrimination analysis) are used in order to 
train a program to recognize images of cats vs dogs.
We perform the task in different steps:
1. Decompose images of dogs and cats into wavelet basis functions to effectively detect the edges.
2. From the wavelet expansion we find the principal components.
3. Desing a statistical discrimination threshold to distinguish between images of dogs and cats. We use LDA.
4. We run the test images to evaluate the accuracy of the model.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#read data from csv and turn them into numpy array
cat_data = pd.read_csv("cat.csv",delimiter=',',header=0)
cat_data = cat_data.values   

dog_data = pd.read_csv("dog.csv",delimiter=',',header=0)
dog_data = dog_data.values

# choose only 60 pictures for analysis and leave the rest 20 for testing
nc = 60
nd = 60
cats = cat_data[:,:nc]
dogs = dog_data[:,:nd]

# show some of the pictures, each column is a picture of size 4096=64*64
fig,ax = plt.subplots(nrows=3,ncols=3,figsize=(10,10))
for i in range(3):
    for j in range(3):
        ax[i,j].imshow(np.reshape(cats[:,i+j],(64,64),'F'))

In [ ]:
# test set shuffled
test = np.hstack((cat_data[:,nc:],dog_data[:,nd:]))
np.random.shuffle(test.T)  # shuffle only columns

# make sure the test is shuffled
fig,ax = plt.subplots(nrows=3,ncols=3,figsize=(10,10))
for i in range(3):
    for j in range(3):
        ax[i,j].imshow(np.reshape(test[:,i+j],(64,64),'F'))

In [ ]:
# function to return wavelet transform of images
from pywt import dwt2

def dc_wavelet(data):
    
    if data.ndim == 1:
        data = data.reshape((data.shape[0],1))
        
    n = data.shape[1]  
    nw = 32*32   # only choose 32 coefficients
    data_wave = np.zeros((nw,n))
    
    data = data.astype(np.float64)
    for i in range(n):
        x = np.reshape(data[:,i],(64,64),'F')
        cA,(cH,cV,cD) = dwt2(x,'haar')
        cod_edge = cH + cV
        data_wave[:,i] = np.ravel(cod_edge)
    return data_wave

# Perform wavelet transform
cats_wave = dc_wavelet(cats)
dogs_wave = dc_wavelet(dogs)

# illustration of original image and its resulting wavelet transform
fig,ax = plt.subplots(nrows=3,ncols=2,figsize=(10,10))
for i in range(3):
        ax[i,0].imshow(np.reshape(cats[:,i],(64,64),'F'))
        ax[i,1].imshow(np.reshape(cats_wave[:,i],(32,32)))

In [ ]:
# function dc_trainer
from scipy.linalg import svd, eig, norm

def dc_trainer(dog0,cat0,nf):
    if dog0.ndim==1:
        dog0 = dog0.reshape(dog0.shape,1)
    nd = dog0.shape[1]
    
    if cat0.ndim==1:
        cat0 = cat0.reshape(cat0.shape,1)
    nc = cat0.shape[1]
    
    U,S,V = svd(np.hstack((dog0,cat0)),full_matrices=False)
    U = U[:,:nf]
    projections = np.diag(S).dot(V)
    dogs = projections[:nf,:nd]
    cats = projections[:nf,nd:nd+nc]
    
    md = np.mean(dogs,axis=1)
    mc = np.mean(cats,axis=1)
    
    sw = np.zeros((nf,nf))
    
    for i in range(nd):
        sw = sw + np.outer(dogs[:,i]-md, dogs[:,i]-md)
        
    for i in range(nc):
        sw = sw + np.outer(cats[:,i]-mc, cats[:,i]-mc)
        
    sb = np.outer(md-mc, md-mc)
    
    d, V2 = eig(sb,sw)
    index = np.argmax(np.abs(d))
    w = V2[:,index]
    w = w/norm(w,2)
    
    vdogs = w.T.dot(dogs)
    vcats = w.T.dot(cats)
    
    result = np.hstack((vdogs,vcats))
    
    if np.mean(vdogs) > np.mean(vcats):
        w = -w
        vdogs = -vdogs
        vcats = -vcats
        
# dog<threshold<cat
    sortdog = np.sort(vdogs)
    sortcat = np.sort(vcats)
    
    t1 = sortdog.size - 1
    t2 = 0
    
    while sortdog[t1]>sortcat[t2]:
        t1 = t1-1
        t2 = t2+1
        
    threshold = (sortdog[t1]+sortcat[t2])/2
    
    return result,w,U,S,V,threshold
            

In [ ]:
# we use a fraction of components (features) to train the model
features = 20
result,w,U,S,V,threshold = dc_trainer(dogs_wave,cats_wave,features)

# performing the analysis on test set
test_wave = dc_wavelet(test)
testmat = U.T.dot(test_wave)
pval = w.dot(testmat)

r = pval>threshold   # cats:True, dogs: False

nrows=np.ceil(np.sqrt(r.sum())).astype(int)
ncols=np.ceil(np.sqrt(r.sum())).astype(int)

# Ideally only cats should be shown
fig,ax = plt.subplots(nrows,ncols,figsize=(10,10))

i = 0
j = 0
for ii in range(test.shape[1]):
    if r[ii]:
        ax[i,j].imshow(np.reshape(test[:,ii],(64,64),'F'))
        if j<ncols-1:
            j = j+1
        else:
            i = i+1
            j=0
                    